<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Perceptron_Multicapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import model_selection
import keras as krs
import tensorflow as tsf
from sklearn.preprocessing import StandardScaler

url1 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv'
url2 = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train = pd.read_csv(url1)
test = pd.read_csv(url2)

train.head(5)

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [44]:
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [45]:
test['character_count'] = test['text'].apply(len)

def word_count(text):
  return len(text.split())
  
test['word_count'] = test['text'].apply(word_count)
test.head(5)

,id,keyword,location,text,character_count,word_count
0,0,NaN,NaN,Just happened a terrible car crash,34,6
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8


In [50]:
data_train=train[['character_count','word_count']]
data_test=test[['character_count','word_count']]

data_target = krs.utils.np_utils.to_categorical(train['target'].values)

scaler = StandardScaler()
scaler.fit(data_train)

data_train=scaler.transform(data_train)
data_test=scaler.transform(data_test)

x_train, x_valid, y_train, y_valid = model_selection.train_test_split(data_train, data_target, test_size=0.3, random_state=0)

model=krs.Sequential()
model.add(krs.layers.normalization.BatchNormalization(input_shape=tuple([x_train.shape[1]])))
model.add(krs.layers.Dense(8, input_dim=2, activation='relu'))
model.add(krs.layers.core.Dropout(rate=0.5))
model.add(krs.layers.normalization.BatchNormalization())
model.add(krs.layers.Dense(8, activation='relu'))
model.add(krs.layers.core.Dropout(rate=0.5))
model.add(krs.layers.Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=100, batch_size=1024, validation_data=(x_valid, y_valid))






Train on 5329 samples, validate on 2284 samples
Epoch 1/100
5329/5329 [==============================] - 0s 83us/step - loss: 0.7873 - accuracy: 0.5171 - val_loss: 0.6839 - val_accuracy: 0.5882
Epoch 2/100
5329/5329 [==============================] - 0s 5us/step - loss: 0.7769 - accuracy: 0.5174 - val_loss: 0.6817 - val_accuracy: 0.5887
Epoch 3/100
5329/5329 [==============================] - 0s 5us/step - loss: 0.7585 - accuracy: 0.5251 - val_loss: 0.6794 - val_accuracy: 0.5863
Epoch 4/100
5329/5329 [==============================] - 0s 5us/step - loss: 0.7577 - accuracy: 0.5247 - val_loss: 0.6774 - val_accuracy: 0.5948
Epoch 5/100
5329/5329 [==============================] - 0s 5us/step - loss: 0.7477 - accuracy: 0.5254 - val_loss: 0.6757 - val_accuracy: 0.5965
Epoch 6/100
5329/5329 [==============================] - 0s 6us/step - loss: 0.7451 - accuracy: 0.5175 - val_loss: 0.6745 - val_accuracy: 0.6014
Epoch 7/100
5329/5329 [==============================] - 0s 5us/step - loss: 0.73

In [52]:
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

5329/5329 [==============================] - 0s 25us/step
Accuracy: 56.44
